# **附录4：关于神经网络 2**

在 1 的基础上，补充记录 **交叉熵（Cross-Entropy）** 损失函数、**Softmax** 输出层激活函数、损失函数的 **正则化（Regularization）** 、权重参数 w 的初始值设定、**超参数（hyper-parameter）** 的选择等内容，依然是基于 **[Neural Networks and Deep Learning 教程](http://neuralnetworksanddeeplearning.com/chap3.html)** 。

> The philosophy is that the best entree to the plethora of available techniques is in-depth study of a few of the most important.（本教程依据的哲学是：入门当下众多技术的最好方式是深入学习其中最重要的那些。）


&emsp;

## **损失函数——交叉熵（Cross-Entropy）**

在 1 中，输出层神经元使用的是 **sigmoid 激活函数**：

$$a=\sigma(z)=\frac{1}{1+e^{-z}}$$

搭配 **二次损失函数（Quadratic Cost Function）**：

$$C = \frac{(y-a)^2}{2}$$

这两者的搭配使用会造成一个问题：学习速率慢，即参数 w 和 b 相对损失函数的变化、变化很慢

$$\frac{\partial C}{\partial w} = (a-y)\sigma'(z)x$$

$$\frac{\partial C}{\partial b} = (a-y)\sigma'(z)$$

这主要是和偏导数中的 $\sigma'(z)$ 有关（当 $\sigma(z)$ 趋近于 1 时，曲线平缓，即导数值 $\sigma '(z)$ 趋近于 0）， 所以引入了 **交叉熵损失函数**，后面会看到，它与 sigmoid 激活函数的搭配使用，会回避掉 $\sigma'(z)$

$$C = - \frac{1}{n} \sum \limits_x [y \ln a + (1-y) \ln(1-a)]$$

$$\frac{\partial C}{\partial w_j} = \frac{1}{n} \sum \limits_x x_j (\sigma(z)-y)$$

$$\frac{\partial C}{\partial b} = \frac{1}{n} \sum \limits_x (\sigma (z) - y)$$

由上面偏导数公式可以看出，使用交叉熵损失函数后，**学习速率的变化主要取决于输出值与预期输出值间的偏差，偏差（即 $\sigma(z)-y$）越大，学习速率越快**

&emsp;

## **输出层激活函数——Softmax**


$$a^L_j = \frac{e^{z^L_j}}{\sum_k e^{z^L_k}}$$


输出层神经元采用 Softmax 作为激活函数后，所有输出结果构成一个 **概率分布**，总和为 1。对比 sigmoid 激活函数，softmax 有两个特点：单调性，输入值 $z^L_j$ 的增大一定会使激活函数的输出值 $a^L_j$ 增大；全局性，神经元的输出值同时还受输出层其他神经元输入值的影响。

$$C \equiv - \ln a^L_y$$

关于 Softmax + Log-likelihood 与 Sigmoid + Cross-Entropy 这两对组合，参考这里的[讨论](https://stats.stackexchange.com/questions/198038/cross-entropy-or-log-likelihood-in-output-layer/445298#445298?newreg=b240ccb1a2a94674b7ce764dc80695a8)

&emsp;

## **过度拟合（overfitting）与正则化（regularization）**

因为神经网络模型中有大量的自由参数（权重 w 和偏差 b），所以对数据的过度拟合会是一个重要问题，过度拟合指的是描述训练数据的模型没有抓到本质，而把训练数据集中因为各种随机因素出现的浮动或偏差等全都考虑在内进行拟合了，结果就是，对训练数据完美拟合，损失函数值很小，但是得到的模型拿到测验数据中测试时的表现并不理想。应对这个问题的方法有：

#### **及早停止（early stopping）**

使用训练数据训练神经网络的同时在 **验证数据（validation data）** 中测验（每一次训练 epoch 结束后），当测验中的表现不再有明显上升趋势的时候即停止训练

关于验证数据的作用，会使用他们来为神经网络的构建确定合适的超参数（hyper-parameter)，然后在测验数据中测试效果，如果使用测验数据选择则可能导致这些参数会对测验数据过度拟合，所以独立于训练数据和测验数据的验证数据很有必要。


#### **增加训练数据量**


#### **正则化——L2 正则**

思路是在损失函数中增加一个正则项：

$$C = -\frac{1}{n}\sum\limits_{x_j}[y_j\ln a^L_j + (1-y_j)\ln(1-a^L_j)]+\frac{\lambda}{2n}\sum\limits_w w^2$$

以上公式就是在交叉熵损失函数的末尾，增加了正则项，$\lambda$ 是正则参数，可以看出正则项与 w 有关，与 b 无关。表达的意思就是要在损失与权重值 w 之间取平衡，如果正则参数值大，偏重要小的参数值，如果正则参数小，就是偏重损失小。

$$\frac{\partial C}{\partial w} = \frac{\partial C_0}{\partial w} + \frac{\lambda}{n} w$$

$$\frac{\partial C}{\partial b} = \frac{\partial C_0}{\partial b}$$

$C_0$ 表示未加正则项的损失函数，损失函数加入正则项， 对反向传播的影响只是 w 相对 C 的变化多了一项 $\frac{\lambda}{n} w$，也好计算，w 和 b 的更新与前面一样：

$$b - \eta\frac{\partial C_0}{\partial b}$$
$$w - \eta\frac{\partial C_0}{\partial w} - \frac{\eta \lambda}{n} w = (1 - \frac{\eta \lambda}{n}) w - \eta \frac{\partial C_0}{\partial w}$$

所以，b 同前，w 也同前，只是按 $1-\frac{\eta \lambda}{n}$ 对它做了收缩，也称**权重衰减（weight decay）**

>  In a nutshell, regularized networks are constrained to build relatively simple models based on patterns seen often in the training data, and are resistant to learning peculiarities of the noise in the training data. The hope is that this will force our networks to do real learning about the phenomenon at hand, and to generalize better from what they learn.

为什么在损失函数中加入正则项会有助于避免过度拟合的问题？因为正则项会约束神经网络习得较小的权重 w，这就意味着，输入数据的变化对输出数据的影响是相对微弱的，神经元们不用对某些具体的误差/噪声大惊小怪，而是要在训练数据集中识别相对稳定的模式（pattern）。较小的参数，意味着更简单的模型。

&emsp;

#### **正则化——L1 正则**

$$C = C_0 + \frac{\lambda}{n} \sum \limits_w |w|$$

L1 与 L2 很相似，从公式可以看出，L1 使用的是 |w|，所以求导后这部分是一个常数，而 L2 是二次项，求导后含有一次项，所以区别就是 L2 对 w 的收缩受 w 本身数值的影响，而 L1 不受（是常数）。于是，如果 w 较大，L2 的收缩程度更大，如果 w 较小，则 L1 的收缩程度更大：

反向传播中 w 的更新变化，使用 L1：

$$w-\frac{\eta \lambda}{n}sgn(w) -\eta \frac{\partial C_0}{\partial w}$$

$sng(w)$ 表示 w 的正负，若为正取 1，若为负取 -1，总之这部分是一个常数，与 w 的数值大小无关，下面 L2 则不同

反向传播中 w 的更新变化，使用 L2：

$$w  - \frac{\eta \lambda}{n} w - \eta\frac{\partial C_0}{\partial w}$$

#### **正则化——dropout**

不改变损失函数，而是调整神经网络的结构，每一个 mini-batch 都 **随机** 使用隐藏层的一半神经元

> "This technique reduces complex co-adaptations of neurons, since a neuron cannot rely on the presence of particular other neurons. It is, therefore, forced to learn more robust features that are useful in conjunction with many different random subsets of the other neurons." In other words, if we think of our network as a model which is making predictions, then we can think of dropout as a way of making sure that the model is robust to the loss of any individual piece of evidence. 

很有意思，就是要限制神经元之间的依赖，使他们尽可能独立稳定地做出稳健的判断

&emsp;

#### **正则化——人为扩充训练数据量**

&emsp;

## **权重初始化**

在 1 中，参数权重 w 的初始值是从平均数为 0、标准差为 1 的正态高斯分布中随机抽取的，这样做存在的问题是由此得到的 z 值（也符合正态分布）标准差过大，z 值的取值范围过大近而使得 sigmoid 激活函数取值趋近于 0 或 1，即神经元饱和，由此导致学习速率很慢，即权重的改变对输出结果的改变微乎其微，最终对损失函数的影响也是非常微弱，这与前面分析 sigmoid 作为输出层激活函数不足的原因相似。所以，相比 0 为均数、1 为标准差，更合理的初始值选取是从 **0 为均值、 $\frac{1}{\sqrt{n_{in}}}$ 为标准差** 的正态分布中随机抽取，也就是收缩高斯分布的范围，避免神经元饱和。

偏差 b 初始值选取还如前，从平均数为 0、标准差为 1 的正态分布中随机抽取。

&emsp;

&emsp;

## **改进后的神经网络**

相比 1，将损失函数替换为交叉熵（Cross-Entropy）、增加正则项、更改权重参数 w 初始值选取

In [2]:
"""
mnist_loader
~~~~~~~~~~~~
这部分代码的功能是以适当的数据结构和格式加载 MNIST 图像数据。这些图片全是手写数字（0-9），要训练的神经网络模型
就是要根据图片判断具体是哪个数字。其中 50000 张图片用作神经网络训练数据集（training set），10000 张用作训练时
的验证数据集（validation set），另还有 10000 张图片用作测试数据集（testing set）。

数据文件 mnist.pkl.gz 存放在仓库内 data 文件夹下。
"""

import pickle
import gzip
import numpy as np

def load_data():
    """
    以元组的形式返回训练数据、验证数据和测试数据
    
    training_data 返回的是由两个元素构成的元组：第一个元素存放的是 50000 张图片数据，即一个由
    50000 个元素构成的数组（numpy ndarry），每一元素又是一个包含 28*28=784 个像素值的数组；元组中的第二个元素也是
    一个由 50000 个元素构成的数组，存放的是每张图片（元组的第一个元素）对应的数字（0-9）
    
    validation_data 以及 test_data 的结构是相同的，只是数量分别都是 10000 个
    
    为了方便神经网络使用，后面 load_data_wrapper 函数又进一步对数据格式做了一点调整

    """
    f = gzip.open('data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return training_data, validation_data, test_data


def load_data_wrapper():
    """
    基于 load_data 函数，返回一个元组，由 (training_data、validation_data、test_data) 三个元素构成，
    更方便后面构建神经网络时使用。
 
    具体地，training_data 是一个列表，包含 50000 个二元元组(x,y)。x 是一个 784 维的数组（numpy.ndarray），存放图片
    数据；y 是一个 10 维的数组，存放表示 x 对应数字的单位向量。
    
    validation_data 以及 test_data 是包含 10000 个二元元组(x,y) 的列表。x 都是一个 784 维的数组，存放图片数据；y 
    对应的则是分类，即 x 对应的数字的值（整数）。
    
    所以，训练数据与验证/测试数据的格式略有不同，都是为了方便构建的神经网络使用。
    """
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    # 下面这三行是从 python2.0 至 python 3.0 做的改动
    test_data = list(test_data)
    training_data = list(training_data)
    validation_data = list(validation_data)
    return training_data, validation_data, test_data

def vectorized_result(j):
    """
    返回一个 10 维的单位向量，j 位放 1.0，其余位置为 0。这是为了将某一个数字（0-9）转换成后面神经网络所需的输出数据。
    即用[1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 表示数字 0，用 [0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0]表示数字 1，依次类推。
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [3]:
training_data, validation_data, test_data = load_data_wrapper()

In [38]:
import numpy as np
import json
import random
import sys

class CrossEntropyCost(object):
    
    @staticmethod
    def fn(a, y):
        """
        如果 a 与 y 都取 1，np.nan_to_num 保证返回值 nan 被转为 0.0
        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    
    @staticmethod
    def delta(z, a, y):
        """
        z 在这里并没有被使用，依然把它作为参数放进来是为了和其他损失函数的 delta 方法的输入参数保持一致
        """
        return (a-y)

class Network(object):
    
    def __init__(self, sizes, cost=CrossEntropyCost):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()      # 更换了参数 w 的初始值选取方法
        self.cost=cost
        
    def default_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y,x)/np.sqrt(x) for x, y in zip(self.sizes[:-1], self.sizes[1:])]
        
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, lmbda=0.0, evaluation_data=None, 
            monitor_evaluation_cost=False, monitor_evaluation_accuracy=False, monitor_training_cost=False,
           monitor_training_accuracy=False):
        """
        evaluation 通常用验证或测验数据，可以 monitor cost 或 accuracy。监测都是在每个 epoch 之后，并返回一个元组，
        元组内包含四个元素，分别是（每个 epoch）测验数据的损失、判别准确率、训练数据的损失、判别准确率
        """
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lmbda, len(training_data))
            print("Epoch %s training complete" % j)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {} ".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data))
            print
        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy
    
    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """
        以一个 mini_batch 为单位，采用梯度下降算法，通过反向传播更新权重 w 和偏差 b。mini_batch 是一个由元组构成的列表，
        元组中存放训练数据(x,y)，eta 是学习速率，lmbda 是正则参数，n 是全部训练数据数
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w - (eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-eta/(len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        """
        返回一个元组(nabla_b, nabla_w)，表示损失函数 C_x 的梯度
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # 正向传播
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # 反向传播
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # 下面部分和 1 略有不同，l = 1 代表最后一层神经元，2 表示倒数第 2 层神经元
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta)*sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def accuracy(self, data, convert=False):
        """
        返回的是输入数据中，神经网络判别正确的数量
        
        如果是验证或测验数据，convert 设为 False，如果是训练数据，设为 True，主要是因为不同数据集内 y 的表征不同，
        而使用不同的表征方式主要是了计算效率
        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y) for (x, y) in data]
            
        return sum(int(x==y) for (x, y) in results)
    
    def total_cost(self, data, lmbda, convert=False):
        """
        如果是训练数据，convert 设为 False；测验或验证数据，设为 True
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights)
        return cost
    
    def save(self, filename):
        data = {"sizes": self.sizes,
                "weights":[w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()
        
def load(filename):
    f = open(filename, "r")
    data = json.load(f)
    f.close
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net
    
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
    
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
    
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


In [39]:
net = Network([784, 30, 10])

In [40]:
net.SGD(training_data, 30, 10, 0.5, lmbda=5.0, evaluation_data = validation_data, monitor_evaluation_cost=True,
       monitor_evaluation_accuracy=True, monitor_training_cost=True, monitor_training_accuracy=True)

Epoch 0 training complete
Cost on training data: 0.4445886341184466 
Accuracy on training data: 47291 / 50000
Cost on evaluation data: 0.7553400146895866
Accuracy on evaluation data: 9480 / 10000
Epoch 1 training complete
Cost on training data: 0.45617955392479187 
Accuracy on training data: 47466 / 50000
Cost on evaluation data: 0.8640032823211206
Accuracy on evaluation data: 9497 / 10000
Epoch 2 training complete
Cost on training data: 0.40752072590103006 
Accuracy on training data: 47909 / 50000
Cost on evaluation data: 0.8745136382374332
Accuracy on evaluation data: 9555 / 10000
Epoch 3 training complete
Cost on training data: 0.4192207749549058 
Accuracy on training data: 47906 / 50000
Cost on evaluation data: 0.9202939857164933
Accuracy on evaluation data: 9534 / 10000
Epoch 4 training complete
Cost on training data: 0.40267494691074485 
Accuracy on training data: 48056 / 50000
Cost on evaluation data: 0.9108077309402147
Accuracy on evaluation data: 9579 / 10000
Epoch 5 training 

([0.7553400146895866,
  0.8640032823211206,
  0.8745136382374332,
  0.9202939857164933,
  0.9108077309402147,
  0.903238007134243,
  0.9487061410139903,
  0.9353531281349307,
  0.9465461000083175,
  0.93293542305162,
  0.9447302527726862,
  0.9139592893055807,
  0.9281116242673739,
  0.9200435905776263,
  0.9272109066490117,
  0.9407555376940431,
  0.9209490298933005,
  0.965810639440988,
  0.9235020161174714,
  0.9292554162713458,
  0.9903825568294475,
  0.9365046952425098,
  0.9393260052811282,
  0.9640146758968597,
  0.9175847117521736,
  0.916637981646649,
  0.9351730767488532,
  0.9522900906612655,
  0.9264775852387415,
  0.9664522987733115],
 [9480,
  9497,
  9555,
  9534,
  9579,
  9587,
  9534,
  9561,
  9585,
  9593,
  9573,
  9636,
  9617,
  9648,
  9617,
  9614,
  9638,
  9542,
  9626,
  9620,
  9519,
  9607,
  9620,
  9574,
  9637,
  9650,
  9616,
  9592,
  9634,
  9583],
 [0.4445886341184466,
  0.45617955392479187,
  0.40752072590103006,
  0.4192207749549058,
  0.402674946